In [4]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

import requests
from dotenv import load_dotenv
import os
load_dotenv()

True

In [ ]:
gpt_turbo = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [ ]:
url = 'https://public-api.meteofrance.fr/public/DPVigilance/v1/textesvigilance/encours'
headers = {
    'accept': '*/*',
    'apikey': os.environ['VIGILENCE_API_KEY']
}
response = requests.get(url, headers=headers)
data = response.json()

In [ ]:
def extract_text_items(data):
    text_items = []
    infos = [
        region for region in data['product']['text_bloc_items'] 
        if len(region['bloc_items']) != 0 and region['bloc_id']=='BULLETIN_DEPARTEMENTAL'
    ]
    for info in infos:
        for bloc_item in info.get('bloc_items', []):
            for text_item in bloc_item.get('text_items', []):
                for subdivision in text_item.get('term_items', []):
                    for subdivision_text in subdivision.get('subdivision_text', []):
                        text_items.append("".join(subdivision_text.get('text', '')))
    return text_items

In [ ]:
infos = list(set(extract_text_items(data)))

In [ ]:
template = """
Fait un résumé concis des informations suivantes : {informations}
"""

In [ ]:
prompt = PromptTemplate.from_template(template)


In [ ]:
chain = (
    {'informations':RunnablePassthrough()} 
    | prompt 
    | gpt_turbo
)

In [ ]:
infos

In [ ]:
answer = chain.invoke(infos)

In [ ]:
print(answer)

In [5]:
from module import *

In [6]:
openai_key = os.environ["OPENAI_API_KEY"]
vigilance_key = os.environ["VIGILENCE_API_KEY"]

In [7]:
news = extract_news(vigilance_key)
llm = gpt_from_api(openai_key)
summary = summarize_news(llm, news)

In [ ]:
llm = gpt_from_api(openai_key)

In [ ]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [ ]:
from langchain_openai import OpenAI